In [ ]:
# python -m pip install gpxpy

'pip' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [5]:
import gpxpy
import pandas as pd
from pathlib import Path

# 1. Notebook 目前所在資料夾 (…\analyze_gpx\GpxClean)
NOTEBOOK_DIR = Path.cwd()

# 2. 專案根目錄 (…\analyze_gpx)
PROJECT_ROOT = NOTEBOOK_DIR.parent

# 3. 原始 GPX 放在 FileClean 裡
FILECLEAN_DIR = PROJECT_ROOT / 'FileClean'

# 4. 定義要處理的資料夾與對應前綴
input_dirs = {
    'Clean_HikingBook_gpx': 'HB',
    'Clean_HikingNote_gpx': 'HN'
}

# 5. 輸出目錄：…\analyze_gpx\GpxClean\gpx_to_csv
output_dir = NOTEBOOK_DIR / 'gpx_to_csv'
output_dir.mkdir(exist_ok=True, parents=True)

# 紀錄若沒 time 欄的檔案
missing_time_files = []

for folder_name, prefix in input_dirs.items():
    folder_path = FILECLEAN_DIR / folder_name
    if not folder_path.exists():
        print(f"資料夾不存在，跳過：{folder_path}")
        continue

    for gpx_path in folder_path.glob('*.gpx'):
        # 解析 GPX
        with open(gpx_path, 'r', encoding='utf-8') as f:
            gpx = gpxpy.parse(f)

        # 收集所有 track point
        points = []
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append({
                        'lat': point.latitude,
                        'lon': point.longitude,
                        'ele': point.elevation,
                        'time': point.time
                    })

        # 轉成 DataFrame
        df = pd.DataFrame(points)

        # ——【功能1】轉 time 欄為 datetime，若真的沒欄位就記錄檔名
        if 'time' not in df.columns:
            missing_time_files.append(gpx_path.name)
        else:
            df['time'] = pd.to_datetime(df['time'])

        # 組出輸出檔名：前綴 + 原檔名（不含副檔名） + .csv
        out_name = f"{prefix}{gpx_path.stem}.csv"
        out_path = output_dir / out_name

        # ——【功能2】如果檔案已存在就跳過
        if out_path.exists():
            print(f"已存在，跳過：{out_name}")
        else:
            df.to_csv(out_path, index=False, encoding='utf-8-sig')
            rel = out_path.relative_to(PROJECT_ROOT)
            print(f"已儲存：{rel}")

# 如果有任何缺少 time 欄的檔案，一次列出
if missing_time_files:
    print("\nWARNING: 以下檔案沒有 `time` 欄位，已跳過轉換：")
    for fname in missing_time_files:
        print(f" - {fname}")

# 最後再印出統計
csv_files = list(output_dir.glob('*.csv'))
print(f"\ngpx_to_csv 資料夾中共有 {len(csv_files)} 筆資料（CSV 檔）")


已儲存：GpxClean\gpx_to_csv\HB0409桃山沒桃子.csv
已儲存：GpxClean\gpx_to_csv\HB0410桃山.csv
已儲存：GpxClean\gpx_to_csv\HB0521單攻桃山.csv
已儲存：GpxClean\gpx_to_csv\HB0829桃山.csv
已儲存：GpxClean\gpx_to_csv\HB1-18桃山.csv
已儲存：GpxClean\gpx_to_csv\HB10-15 桃山單攻.csv
已儲存：GpxClean\gpx_to_csv\HB1091011-桃山.csv
已儲存：GpxClean\gpx_to_csv\HB1091226桃山單攻.csv
已儲存：GpxClean\gpx_to_csv\HB111.09.25桃山單攻.csv
已儲存：GpxClean\gpx_to_csv\HB111.8.22桃山單攻.csv
已儲存：GpxClean\gpx_to_csv\HB1110115桃山⛰️.csv
已儲存：GpxClean\gpx_to_csv\HB112-9-24桃山單攻.csv
已儲存：GpxClean\gpx_to_csv\HB12.03桃山.csv
已儲存：GpxClean\gpx_to_csv\HB20123-9-23爬桃山爬到厭世.csv
已儲存：GpxClean\gpx_to_csv\HB2014-5-25單攻桃山.csv
已儲存：GpxClean\gpx_to_csv\HB20200418 桃山.csv
已儲存：GpxClean\gpx_to_csv\HB202011 桃山單攻.csv
已儲存：GpxClean\gpx_to_csv\HB20201212 桃山.csv
已儲存：GpxClean\gpx_to_csv\HB2021-11-25 武陵桃山上行.csv
已儲存：GpxClean\gpx_to_csv\HB2021-11-28 桃山.csv
已儲存：GpxClean\gpx_to_csv\HB2021-11-7 桃山.csv
已儲存：GpxClean\gpx_to_csv\HB2021-12-11 桃山.csv
已儲存：GpxClean\gpx_to_csv\HB2021-12-20 桃山單攻.csv
已儲存：GpxClean\gpx_to_csv\HB2021-9-

### 水平距離計算

以簡化平面距離進行計算（適合小範圍，緯度變化不大，假設地球為平面
111320 是一個近似值，代表"緯度"每度約等於 111320 公尺
地球周長在赤道最大，越靠近極地越小，需要用 cos 隨緯度修正

緯度差，換算成公尺
dy = (lat2 - lat1) * 111320
經度差，換算成公尺
dx = (lon2 - lon1) * 111320 * cos(radians((lat1 + lat2) / 2))

畢氏定理
distance = sqrt(dx**2 + dy**2)
return distance



In [8]:
import pandas as pd
from math import cos, radians, sqrt
from pathlib import Path

def flat_distance(lat1, lon1, lat2, lon2):
    dy = (lat2 - lat1) * 111320
    dx = (lon2 - lon1) * 111320 * cos(radians((lat1 + lat2) / 2))
    return sqrt(dx**2 + dy**2)

NOTEBOOK_DIR = Path.cwd()
CSV_DIR      = NOTEBOOK_DIR / 'gpx_to_csv'
ADD_DIR      = NOTEBOOK_DIR / 'add_cum_csv'
ADD_DIR.mkdir(exist_ok=True, parents=True)

summaries     = []
skipped_files = []   # ← 用來記錄所有跳過的檔案

for csv_path in CSV_DIR.glob('*.csv'):
    out_path = ADD_DIR / csv_path.name
    # 1) 已存在就跳過
    if out_path.exists():
        skipped_files.append(csv_path.name)
        print(f"已存在，跳過：{csv_path.name}")
        continue

    # 2) 空檔、無 time 欄都跳過
    try:
        df = pd.read_csv(csv_path)
    except pd.errors.EmptyDataError:
        skipped_files.append(csv_path.name)
        print(f"警告：檔案為空 → {csv_path.name}")
        continue

    if 'time' not in df.columns:
        skipped_files.append(csv_path.name)
        print(f"警告：缺少 time 欄 → {csv_path.name}")
        continue

    # 強制轉 datetime，並去除轉換失敗的列
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    df = df.dropna(subset=['time']).sort_values('time').reset_index(drop=True)
    if df.empty:
        skipped_files.append(csv_path.name)
        print(f"警告：無有效時間資料 → {csv_path.name}")
        continue

    # 計算累積欄位
    cum_time = []; cum_dist = []; cum_up = []; cum_down = []
    total_dist = total_up = total_down = 0.0
    start_time = df.loc[0, 'time']
    last_lat, last_lon, last_ele = df.loc[0, ['lat','lon','ele']]

    for idx, row in df.iterrows():
        cum_time.append(row['time'] - start_time)
        if idx > 0:
            d = flat_distance(last_lat, last_lon, row['lat'], row['lon'])
            total_dist += d
            delta_ele = row['ele'] - last_ele
            if delta_ele > 0: total_up += delta_ele
            else:             total_down += abs(delta_ele)
        cum_dist.append(total_dist)
        cum_up.append(total_up)
        cum_down.append(total_down)
        last_lat, last_lon, last_ele = row['lat'], row['lon'], row['ele']

    df['cum_dist'] = cum_dist
    df['cum_up'] = cum_up
    df['cum_down'] = cum_down
    df['cum_time'] = cum_time

    df.to_csv(out_path, index=False, encoding='utf-8-sig')
    print(f"已另存：{out_path.relative_to(NOTEBOOK_DIR)}")

    summaries.append({
        '檔案名':            csv_path.name,
        'max累計水平距離':  cum_dist[-1],
        'max累計爬升高度':  cum_up[-1],
        'max累計下降高度':  cum_down[-1],
        'max累計花費時間':  cum_time[-1],
    })

# 輸出 summary.csv
if summaries:
    pd.DataFrame(summaries).to_csv(ADD_DIR / 'summary.csv', index=False, encoding='utf-8-sig')

# 最後列印所有跳過的檔案
if skipped_files:
    print("\n===== 跳過的檔案清單 =====")
    for name in skipped_files:
        print(f"• {name}")
else:
    print("\n沒有任何檔案被跳過。")


已另存：add_cum_csv\HB0409桃山沒桃子.csv
已另存：add_cum_csv\HB0410桃山.csv
已另存：add_cum_csv\HB0521單攻桃山.csv
已另存：add_cum_csv\HB0829桃山.csv
已另存：add_cum_csv\HB1-18桃山.csv
已另存：add_cum_csv\HB10-15 桃山單攻.csv
已另存：add_cum_csv\HB1091011-桃山.csv
已另存：add_cum_csv\HB1091226桃山單攻.csv
已另存：add_cum_csv\HB111.09.25桃山單攻.csv
已另存：add_cum_csv\HB111.8.22桃山單攻.csv
已另存：add_cum_csv\HB1110115桃山⛰️.csv
已另存：add_cum_csv\HB112-9-24桃山單攻.csv
已另存：add_cum_csv\HB12.03桃山.csv
已另存：add_cum_csv\HB20123-9-23爬桃山爬到厭世.csv
已另存：add_cum_csv\HB2014-5-25單攻桃山.csv
已另存：add_cum_csv\HB20200418 桃山.csv
警告：檔案為空 → HB202011 桃山單攻.csv
已另存：add_cum_csv\HB20201212 桃山.csv
已另存：add_cum_csv\HB2021-11-25 武陵桃山上行.csv
已另存：add_cum_csv\HB2021-11-28 桃山.csv
已另存：add_cum_csv\HB2021-11-7 桃山.csv
已另存：add_cum_csv\HB2021-12-11 桃山.csv
已另存：add_cum_csv\HB2021-12-20 桃山單攻.csv
已另存：add_cum_csv\HB2021-9-26 桃山單攻.csv
已另存：add_cum_csv\HB20210320_桃山單攻.csv
已另存：add_cum_csv\HB20210927桃山單攻.csv
已另存：add_cum_csv\HB20211016-桃山單攻.csv
已另存：add_cum_csv\HB2022-10-08 桃山.csv
已另存：add_cum_csv\HB2022-6-19 桃山.csv
已另存：add_

C:\Users\Pin\AppData\Local\Temp\ipykernel_17840\2703390695.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'], errors='coerce')


已另存：add_cum_csv\HN桃山單攻 (25).csv
已另存：add_cum_csv\HN桃山單攻 (26).csv
已另存：add_cum_csv\HN桃山單攻 (27).csv
已另存：add_cum_csv\HN桃山單攻 (28).csv
已另存：add_cum_csv\HN桃山單攻 (29).csv
已另存：add_cum_csv\HN桃山單攻 (3).csv
已另存：add_cum_csv\HN桃山單攻 (30).csv
已另存：add_cum_csv\HN桃山單攻 (31).csv
已另存：add_cum_csv\HN桃山單攻 (32).csv
已另存：add_cum_csv\HN桃山單攻 (33).csv
已另存：add_cum_csv\HN桃山單攻 (34).csv
已另存：add_cum_csv\HN桃山單攻 (35).csv
已另存：add_cum_csv\HN桃山單攻 (36).csv
已另存：add_cum_csv\HN桃山單攻 (37).csv
已另存：add_cum_csv\HN桃山單攻 (38).csv
已另存：add_cum_csv\HN桃山單攻 (39).csv
已另存：add_cum_csv\HN桃山單攻 (4).csv
已另存：add_cum_csv\HN桃山單攻 (40).csv
已另存：add_cum_csv\HN桃山單攻 (41).csv
已另存：add_cum_csv\HN桃山單攻 (42).csv
已另存：add_cum_csv\HN桃山單攻 (43).csv
已另存：add_cum_csv\HN桃山單攻 (44).csv
已另存：add_cum_csv\HN桃山單攻 (45).csv
已另存：add_cum_csv\HN桃山單攻 (46).csv
已另存：add_cum_csv\HN桃山單攻 (47).csv
已另存：add_cum_csv\HN桃山單攻 (48).csv
已另存：add_cum_csv\HN桃山單攻 (49).csv
已另存：add_cum_csv\HN桃山單攻 (5).csv
已另存：add_cum_csv\HN桃山單攻 (50).csv
已另存：add_cum_csv\HN桃山單攻 (6).csv
已另存：add_cum_csv\HN桃山單攻 (7).csv
已另存：add_cum_c

In [ ]:
# 536 筆資料中，有 9 筆異常，剩下527 筆
# 統計 add_cum_csv 資料夾中 CSV 檔案數
added_files = list(ADD_DIR.glob('*.csv'))
print(f"add_cum_csv 資料夾中共有 {len(added_files)} 筆 CSV 檔")

add_cum_csv 資料夾中共有 527 筆 CSV 檔
